# Computing PIV uncertainty

In [ ]:
from h5rdmtoolbox.h5wrapper import H5PIV
from h5rdmtoolbox import tutorial
import numpy as np

Get an example HDF filename from the ILA vortex pair example (https://www.pivtec.com/pivview.html):

To compute the uncertainty of a PIV measurement, we need to gather some specific datasets, namely the at minimum the pixel coordinates, the displacements and the raw images. The class property `UncertaintyDataset` does this for us. Calling it will return a `xarray.Dataset` with the displacement variables.

Let's load the vortex example and fetch image A and imabe B:

In [ ]:
with tutorial.get_H5PIV('vortex_snapshot', 'r+') as h5:
    disp = h5.DisplacementVector[:,:]
    imgA = h5.imgA[:,:]
    imgB = h5.imgB[:,:]
disp

The uncertainty dataset has the coordinates `x` and `y`, the displacements arrays `dx` and `dy` but also the pixel coordinates `ix` and `iy`

Next, let's create a more or less random uncertainty method. In this example we do not compute the real error but assume one, just to explain the workflow of cumputing the uncertainty from the dataset:

In [ ]:
def my_uncertainty_method(uds, imgA, imgB):
    """
    Dummy uncertainty method for this tutorial.
    Returns the same dataset but with added uncertainties
    
    Parameters
    ----------
    uds: XRUncertaintyDataset
        The uncertainty dataset containing, x, y, ix, iy, dx, dy, ...
    imgA: np.ndarray
        2d PIV image A. Will not be touch in this example
    imgB: np.ndarray
        2d PIV image B. Will not be touch in this example
        
    Returns
    -------
    uds: XRUncertaintyDataset    
    """
    import xarray as xr
    xerr = 0.05
    yerr = 0.075
    udx = np.abs(uds.dx)*xerr
    uds['udx'] = xr.DataArray(dims=uds.dx.dims, data=udx,
                                        attrs={'standard_name': f'uncertainty_of_{uds.dx.standard_name}',
                                               'units': 'pixel',
                                               'piv_uncertainty_method': 'my_uncertainty_method'})
    udy = np.abs(uds.dy)*yerr
    uds['udy'] = xr.DataArray(dims=uds.dy.dims, data=udy,
                                        attrs={'standard_name': f'uncertainty_of_{uds.dy.standard_name}',
                                               'units': 'pixel',
                                               'piv_uncertainty_method': 'my_uncertainty_method'})
    return uds

In [ ]:
type(disp)

In [ ]:
un = disp.compute_uncertainty(my_uncertainty_method, imgA, imgB)

The `XRUncertaintyDataset` not got some more datasets and the uncertainty `DataArray` `delta_dx` and `delta_dy`

Now, let's plot the magnitude of the displacements:

In [ ]:
un.compute_magnitude()
_ = un.magnitude[:].plot.contourf(vmax=6, vmin=0)

Let's have a look at the `udx` dataset of the error without the extreme values which may be wrong:

In [ ]:
# un.get_by_attribute('standard_name')

In [ ]:
# un.get_by_standard_name('uncertainty_of_x_displacement')

In [ ]:
udx = un.get_by_standard_name('uncertainty_of_x_displacement')
_ = udx.where(np.abs(udx) < 20).plot.contourf()
print(f'Error in x-direction: {udx.mean().values}')
print(f'Absolute relative error in x-direction: {np.divide(udx, np.abs(un.dx)).mean().values}')

In [ ]:
with H5PIV(h5.hdf_filename, 'r+') as h5:
    h5.create_group('uncertainty', overwrite=True)
    h5['uncertainty'].create_dataset('delta_dx', data=un.get_by_standard_name('uncertainty_of_x_displacement'), overwrite=True)
    h5['uncertainty'].create_dataset('delta_dy', data=un.get_by_standard_name('uncertainty_of_x_displacement'), overwrite=True)

Let's write the data into the HDF file:

In [ ]:
with H5PIV(h5.hdf_filename, 'r+') as h5:
    h5.dump()
    h5.uncertainty.delta_dx[:,:].plot()